<h1 style="font-family:Impact,Arial;font-size:30px;">37005 Fundamentals of Derivative Security Pricing - Spring 2025</h1>
<h1 style="font-family:Impact,Arial;font-size:45px;">Assignment Part 2</h1>
<h2 style="font-family:Arial;">Erik Schl&ouml;gl</h2>
<p><small> School of Mathematical &amp; Physical Sciences<br>
University of Technology Sydney
</small></p>
<p>
<a href="mailto:Erik.Schlogl@uts.edu.au?Subject=37000 JIT" target="_blank">
<small><font color=MediumVioletRed>Erik.Schlogl@uts.edu.au</font></small></a>
</p>
<hr style="height:5px;border:none;color:#333;background-color:#333;" />

# Standard normal CDF

## Task 1
Write a program in Python that asks a user for a number `x`, and then outputs the value of the cumulative distribution function of the standard normal distribution at `x`. *(1 mark)*
## Example

`Enter x: 0.3
The value of the standard normal CDF at 0.3 is 0.6179114221889526`

**Hint:** You'll want to use the `input(...)` and `print(...)` functions for this. Recall that `input(...)` returns a string, which must be converted to a floating point number using `float(...)`. If we do `from scipy.stats import norm`, then the cumulative distribution function of the standard normal distribution can be accessed as `norm.cdf(...)`.

In [1]:
from scipy.stats import norm

x = float(input("Enter x: "))
print("The value of the standard normal CDF at", x, "is", norm.cdf(x))

The value of the standard normal CDF at 0.3 is 0.6179114221889526


# Black/Scholes formula in Python
The Black/Scholes price of a European call option expiring at time $T$ with strike price $K$ is
$$
C(S,t)=SN(d_1)−Ke^{−r(T−t)}N(d_2)
$$
where $S$ is the current price of the underlying asset, $t$ the current time and $r$ the continuously compounded riskfree interest rate. $N(d)$ denotes the cumulative distribution function of the standard normal distribution, and
$$
\begin{aligned}
d_1 &= \frac{\ln\frac{S}K+(r+\frac12\sigma^2)(T−t)}{\sigma\sqrt{T-t}}\\
d_2 &= d_1−\sigma\sqrt{T-t}
\end{aligned}
$$
where the $\sigma$ denotes the volatility of the underlying asset.

Similarly, the price of a European put option expiring at time $T$ with strike price $K$ is
$$
P(S,t)=Ke^{−r(T−t)}N(−d_2)−SN(−d_1)
$$
## Task 2
Using the scaffold provided, write a Python function which calculates the Black/Scholes price of the option, where the function takes six arguments (in this order): $S$, $K$, $\sigma$, $r$, $T$ and a 1 for a call or -1 for a put. <em>(2 marks)</em>

## Example:

`Enter the underlying stock price: 100
Enter the strike price: 100
Enter the volatility: 0.3
Enter continuously compounded interest rate: 0.05
Enter the time to maturity: 2
Enter 1 for call or -1 for put option: 1
The option price is: 21.193735255280203`

## Another example (a put option):

`Enter the underlying stock price: 100
Enter the strike price: 100
Enter the volatility: 0.3
Enter continuously compounded interest rate: 0.05
Enter the time to maturity: 2
Enter 1 for call or -1 for put option: -1
The option price is: 11.677477058876157`

In [2]:
import numpy as np


def BlackScholes(S, K, sgm, r, T, callput):
    d1 = (((sgm**2) / 2 + r) * T + np.log(S / K)) / (sgm * np.sqrt(T))
    d2 = d1 - sgm * np.sqrt(T)
    option_price = callput * (
        S * norm.cdf(callput * d1) - K * np.exp(-r * T) * norm.cdf(callput * d2)
    )
    return option_price


stock = float(input("Enter the underlying stock price: "))
strike = float(input("Enter the strike price: "))
sigma = float(input("Enter the volatility: "))
interest = float(input("Enter continuously compounded interest rate: "))
maturity = float(input("Enter the time to maturity: "))
callput = int(input("Enter 1 for call or -1 for put option: "))
print("The option price is: ")
print(BlackScholes(stock, strike, sigma, interest, maturity, callput))

The option price is: 
17.80110375541804


# Monte Carlo simulation
The goal is to calculate the expected value of a function $f(\cdot)$ of a random variable $x$, where the distribution of $x$ is given by the probability density $\psi(x)$, i.e.
$$
E[f(x)]=\int_{-\infty}^{\infty}f(x)\psi(x)dx
$$

## Outline of the Monte Carlo simulation
1. Establish a procedure for drawing variates $x$ from the target distribution $\psi(x)$.
2. Initialise the variables:
   RunningSum = 0 
   RunningSumSquared = 0 
   $i=1$
3. Draw a realisation $x_i$ from the target distribution.
4. Add $f(x_i)$ to RunningSum and  $(f(x_i))^2$ to RunningSumSquared.
5. Increment the counter $i$. If $i$ is less than the maximum number of iterations, go to step 3.
6. Calculate the simulated mean by dividing RunningSum by the total number of iterations.
7. Calculate the variance of the simulations by dividing RunningSumSquared by the total number of iterations and subtracting the square of the mean.

## Error estimation for Monte Carlo methods
By the Central Limit Theorem, we know that for a large number $N$ of simulations, the simulation mean $X_N$ is approximately normally distributed, with standard deviation
$$
\sqrt{\frac{\sigma^2}N}
$$
where the simulation variance is an estimate for $\sigma^2$.

Thus, if there is no bias, the simulation mean is normally distributed around the target value with a standard deviation, which decreases with $\sqrt{N}$.

A 95% confidence interval for the target value is therefore approximately given by
$$
\left[X_N-2\sqrt{\frac{\sigma^2}N};X_N+2\sqrt{\frac{\sigma^2}N}\right]
$$

<font color='red'>**Monte Carlo simulation without error bounds is meaningless!**</font>

The NumPy function `random.standard_normal()` returns a random variate drawn from the standard normal distribution, while `random.standard_normal(n)` returns `n` such variates in a Numpy array:

In [3]:
import numpy as np

print(np.random.standard_normal())
n = 5
print(np.random.standard_normal(n))

2.795175740710806
[ 0.19636222  1.0371702  -1.00142738  1.02512038  0.05607444]


Recall that a standard normal random variable can be converted into a normal random variable of desired mean and standard deviation by multiplying by the standard deviation and adding the mean.

## Monte Carlo pricing of a Black/Scholes call option
In the Black/Scholes model, the price of the underlying stock follows Geometric Brownian motion, with the dynamics under the risk-neutral measure given by
$$S(T)=S(t)\exp\left\{\left(r−\frac12\sigma^2\right)(T−t)+\sigma(W(T)−W(t))\right\}$$
Recall that the time 0 price of a European call option (and analogously the put option) expiring at time $T$ with strike price $K$ can be expressed as the expectation under the risk-neutral measure of 
$$C=E\left[e^{−rT}\max(0,S(T)−K)\right]$$
Thus we can write a Python function which calculates the Monte Carlo estimate `MC` for the Black/Scholes price of the option and the standard deviation `MCstd` of the simulation mean, where the function takes seven arguments (in this order): $S$, $K$, $\sigma$, $r$, $T$, a 1 for a call or -1 for a put, and $n$, the number of sampling iterations of the Monte Carlo algorithm:

In [14]:
def BlackScholesMC(S, K, sgm, r, T, callput, n):
    w = np.random.standard_normal(n)
    ST = S * np.exp((r - 0.5 * sgm**2) * T + sgm * np.sqrt(T) * w)
    payoff = callput * np.maximum(ST - K, 0)
    MC = np.exp(-r * T) * np.mean(payoff)
    MCstd = np.exp(-r * T) * np.std(payoff) / np.sqrt(n)
    return MC, MCstd

To run this code with user inputs:

In [15]:
stock = float(input("Enter the underlying stock price: "))
strike = float(input("Enter the strike price: "))
sigma = float(input("Enter the volatility: "))
interest = float(input("Enter continuously compounded interest rate: "))
maturity = float(input("Enter the time to maturity: "))
callput = int(input("Enter 1 for call or -1 for put option: "))
n = int(input("Enter the number of simulations: "))
MC, MCstd = BlackScholesMC(stock, strike, sigma, interest, maturity, callput, n)
print("The MC estimate for the option price is: ")
print(MC)
print("The 2 standard deviation confidence interval for the option price is: ")
print(MC - 2 * MCstd, MC + 2 * MCstd)

The MC estimate for the option price is: 
17.77766747554084
The 2 standard deviation confidence interval for the option price is: 
17.10188617042339 18.45344878065829


## Task 3

Consider a European call option with maturity $T$ on a foreign asset $\tilde S$, which pays (in domestic currency) at time $T$
$$
\max[0,X(T)\tilde S(T)-X(0)\tilde S(T)]
$$
Assume that $\tilde S$ follows the dynamics (under the foreign risk--neutral measure) of
$$
d\tilde S(t) = \tilde S(t)(\tilde rdt+\xi d\tilde W(t))
$$
and the exchange rate $X$ (in units of domestic currency per unit of foreign currency) follows the dynamics (under the domestic risk--neutral measure) of
$$
dX(t) = X(t)((r-\tilde r)dt+\sigma dW(t))
$$
with $W(t)$ and $\tilde W(t)$ two-dimensional standard Brownian motions under the respective measures, $r$ and $\tilde r$ scalar constants, and $\xi$ and $\sigma$ are two--dimensional constant vectors.
### Task 3(a)
Using the scaffold provided, write a Python function which calculates the price of the option, where the function takes eight arguments (in this order): $\tilde S(0)$, $X(0)$, $r$, $\tilde r$, $T$, the Black/Scholes volatility of $\tilde S$, the Black/Scholes volatility of $X$ and the correlation coefficient between the dynamics of $\tilde S$ and $X$. <em>(9 marks)</em>

In [13]:
def Task3Option(S, X, r, r_f, T, volS, volX, rho):
    d1 = (((volX**2) / 2 + r - r_f + rho * volS * volX) * T) / (volX * np.sqrt(T))
    d2 = d1 - volX * np.sqrt(T)
    option_price = (
        S
        * X
        * (norm.cdf(d1) - np.exp((r_f - r - rho * volS * volX) * T) * norm.cdf(d2))
    )
    return option_price


Task3Option(100, 1.2, 0.05, 0.03, 1, 0.2, 0.1, 0.5)

np.float64(6.698252581126463)

### Task 3(b)
Calculate the price of this option by Monte Carlo simulation and use this to verify that your implementation of Task 3(a) is correct. *(12 marks)*

In [9]:
def Task3bOption(S, X, r, r_f, T, volS, volX, rho, num_sim):
    w1 = np.random.standard_normal(num_sim)
    w2 = rho * w1 + np.sqrt(1 - rho**2) * np.random.standard_normal(num_sim)
    ST = S * np.exp((r_f - 0.5 * volS**2) * T + volS * np.sqrt(T) * w1)
    XT = X * np.exp((r - r_f - 0.5 * volX**2) * T + volX * np.sqrt(T) * w2)
    payoff = ST * np.maximum(XT - X, 0)
    MC = np.exp(-r * T) * np.mean(payoff)
    MCstd = np.exp(-r * T) * np.std(payoff) / np.sqrt(num_sim)
    return MC, MCstd


MC, MCstd = Task3bOption(100, 1.2, 0.05, 0.03, 1, 0.2, 0.1, 0.5, 10000)
print(
    "The MC estimate for the option price is: ",
    MC - norm.ppf(0.975) * MCstd,
    " to ",
    MC + norm.ppf(0.975) * MCstd,
)

The MC estimate for the option price is:  6.62319641875729  to  7.01088112335958
